In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../TWO_CENTURIES_ANALYSIS_CLEAN_WITH_PACE.csv')

In [ ]:
variables = [
    'Distance_km',
    'Avg_speed_kmh',
    'Athlete age',
    'Time_hours',
    'Speed_calc_kmh',
    'Speed_diff',
    'Pace_min_per_km'
]

# Usaremos 3 semillas para generar muestras distintas
semillas = [42, 99, 123]
resultados = []

# Preprocesamiento: seleccionar variables y escalar
def preparar_datos(df_sample):
    df_sel = df_sample[variables].dropna()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_sel)
    return X_scaled, df_sel

# Aplicación de agrupamiento jerárquico aglomerativo y métricas
def aplicar_jerarquico(X, n_clusters=4, linkage='ward'):
    modelo = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    etiquetas = modelo.fit_predict(X)
    sil = silhouette_score(X, etiquetas)
    calinski = calinski_harabasz_score(X, etiquetas)
    davies = davies_bouldin_score(X, etiquetas)
    return etiquetas, sil, calinski, davies

# Reducción a 2 dimensiones y visualización de los clusters
def graficar_clusters(X, etiquetas, titulo):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=etiquetas, palette='tab10', s=10)
    plt.title(titulo)
    plt.xlabel("PCA 1")
    plt.ylabel("PCA 2")
    plt.legend(title="Cluster", loc="best", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

# Proceso para las 3 muestras
for i, seed in enumerate(semillas):
    print(f"Procesando muestra {i+1} con semilla {seed}...")
    df_muestra = df.sample(n=50000, random_state=seed)
    X_scaled, df_sel = preparar_datos(df_muestra)
    etiquetas, sil, calinski, davies = aplicar_jerarquico(X_scaled, n_clusters=4, linkage='ward')
    resultados.append({
        'Muestra': i+1,
        'Silhouette': sil,
        'Calinski-Harabasz': calinski,
        'Davies-Bouldin': davies
    })
    graficar_clusters(X_scaled, etiquetas, f"Jerárquico (ward) - Muestra {i+1}")

# Mostrar resultados comparativos
pd.DataFrame(resultados)